In [ ]:
import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import requests
from datetime import datetime 
import time
import pytz
import os

In [27]:
cat_history_df = pd.read_csv('cat_history.csv')

In [28]:
cat_history_df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,url,name,age,gender,cafe_cat,kid_approved,dog_approved,companion_cat,bonded_pair,single_cat,type,description,system_import_at
0,0,0.0,0.0,18576198,https://catcafebk.com/our-cats/?cat=18576198,Aeon,10 Years 5 Months,Female,False,False,False,True,False,True,Domestic Short Hair,"Aeon is a gorgeous, chatty tortie girl with a ...",2023-01-22 19:13:48+00:00
1,1,1.0,1.0,18949108,https://catcafebk.com/our-cats/?cat=18949108,Aioli & Dijon,4 Months 3 Days,NaN,False,True,True,False,True,False,Domestic Short Hair,Aioli & Dijon are full of zest and zing — whet...,2023-01-22 19:13:48+00:00
2,2,2.0,2.0,18049093,https://catcafebk.com/our-cats/?cat=18049093,Apollo & Artemis,1 Year 1 Month,NaN,False,True,True,False,True,False,Domestic Short Hair / Mixed,Apollo and Artemis are sweet and sassy teens r...,2023-01-22 19:13:48+00:00
3,3,3.0,3.0,18844592,https://catcafebk.com/our-cats/?cat=18844592,Baby Shark,9 Months 7 Days,Female,False,False,False,False,False,False,Domestic Short Hair,Baby Shark is a sweet tuxedo who likes taking ...,2023-01-22 19:13:48+00:00
4,4,4.0,4.0,18949086,https://catcafebk.com/our-cats/?cat=18949086,Bada & Pow,4 Months 19 Days,Male,False,True,True,False,True,False,Domestic Short Hair,"Bada and Pow are two adorable, playful brother...",2023-01-22 19:13:48+00:00


In [29]:
cat_history_df.system_import_at.unique()

array(['2023-01-22 19:13:48+00:00', '2023-01-22 19:49:42+00:00',
       '2023-01-22 19:51:39+00:00'], dtype=object)

In [30]:
cat_history_df.shape

(352, 17)

In [32]:
last_mask = (cat_history_df['system_import_at'] == '2023-01-22 19:49:42+00:00')
this_mask = (cat_history_df['system_import_at'] == '2023-01-22 19:51:39+00:00')

In [37]:
last_cats = cat_history_df[last_mask]['name'].unique()
this_cats = cat_history_df[this_mask]['name'].unique()

diff = [x for x in this_cats if x not in last_cats]
print(diff) 

['Ernutter & Andolf']
